In [17]:
import yaml
from yaml.loader import SafeLoader
from datetime import datetime
import time
import pandas as pd
fname='Milestone1B.yaml'

with open(fname) as f:
    conf = yaml.load(f, Loader=SafeLoader)
print(conf)

{'M1B_Workflow': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskA': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}, 'FlowA': {'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activ

In [18]:
def TimeFunction(FunctionInput,ExecutionTime):
    print(1)
    time.sleep(int(ExecutionTime))
    print(int(ExecutionTime))
    return
    
def DataLoad(Filename):
    print(2)
    DataTable=pd.read_csv(Filename)
    NoOfDefects=DataTable.shape[0]-1    
    return DataTable, NoOfDefects

def callfunction(name,inputs):
    if name=='TimeFunction':
        TimeFunction(inputs[0],inputs[1])
    elif name=='DataLoad':
        DataLoad(inputs[0])

In [21]:
import copy
import threading
def runTask(task,b_list):
    print(task)
    name=''
    for i in b_list:
        name+=str(i)+'.'
    
    f.write(str(datetime.now())+';'+name[:-1]+' Entry\n')
    if task['Type']=='Task':
        funs=''
        params=''
        inputs=[]

        for i in task['Inputs'].values():
            inputs.append(i)
            funs+=str(i)+', '
        params=funs
                
        
        f.write(str(datetime.now())+';'+name[:-1]+' Executing ' +str(task['Function'])+' ('+params[:-2]+')\n')
        callfunction(task['Function'],inputs)
        
    else:
        if(task['Execution']=='Sequential'):
            Act=task['Activities']
            for i in Act:
                par=copy.deepcopy(b_list)
                par.append(i)
                runTask(Act[i],par)
        else:
            Act=task['Activities']
            threads=[]
            for i in Act:
                par=copy.deepcopy(b_list)
                par.append(i)
                t=threading.Thread(target=runTask, args=(Act[i],par,))
                threads.append(t)
                t.start()
            for t in threads:
                t.join()
    f.write(str(datetime.now())+';'+name[:-1]+' Exit\n')

In [22]:
Task_list = list(conf.keys())[0]
f = open(fname[:-5]+"_Log.txt", "w")
f.write(str(datetime.now())+';'+Task_list+' Entry\n')
Activities_list = conf[Task_list]['Activities']
for i in Activities_list:
    runTask(Activities_list[i],[Task_list,i])

f.write(str(datetime.now())+';'+Task_list+' Exit')
f.close()

{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}
1
1
{'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskG': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 